In [8]:
import cv2
import os

def generate_dataset():
    # Ensure the cascade file exists
    cascade_path = "haarcascade_frontalface_default.xml"
    if not os.path.exists(cascade_path):
        print(f"Error: Cascade file '{cascade_path}' not found.")
        return
    
    face_classifier = cv2.CascadeClassifier(cascade_path)
    
    # Ensure data directory exists
    if not os.path.exists("data"):
        os.makedirs("data")
    
    def face_cropped(img):
        if img is None:
            return None
        
        try:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_classifier.detectMultiScale(gray, 1.3, 5)
            
            if len(faces) == 0:  # Fixed: use len() instead of comparing with ()
                return None
                
            for (x, y, w, h) in faces:
                cropped_face = img[y:y+h, x:x+w]
                return cropped_face
                
            return None
        except Exception as e:
            print(f"Error in face_cropped: {e}")
            return None
    
    # Try different camera indices
    camera_index = 0
    max_attempts = 3
    
    for i in range(max_attempts):
        print(f"Trying camera index {camera_index}")
        cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW)  # Use DirectShow on Windows
        
        if cap.isOpened():
            print(f"Successfully opened camera with index {camera_index}")
            # Wait for camera to initialize
            import time
            time.sleep(2)
            ret, test_frame = cap.read()
            if ret and test_frame is not None:
                print("Camera providing valid frames")
                break
        
        print(f"Failed with camera index {camera_index}")
        cap.release()
        camera_index += 1
        
        if i == max_attempts - 1:
            print("Could not initialize any camera")
            return
    
    id = 1
    img_id = 0
    
    while True:
        ret, frame = cap.read()
        
        if not ret or frame is None:
            print("Failed to capture frame")
            break
            
        face = face_cropped(frame)
        
        if face is not None:
            img_id += 1
            face = cv2.resize(face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = f"data/user.{id}.{img_id}.jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            
            cv2.imshow("Cropped face", face)
            
            # Show the original frame with rectangle
            cv2.rectangle(frame, (frame.shape[1]//2-100, frame.shape[0]//2-100), 
                         (frame.shape[1]//2+100, frame.shape[0]//2+100), (0, 255, 0), 2)
            cv2.imshow("Video Frame", frame)
        
        if cv2.waitKey(1) == 13 or img_id >= 200:  # 13 is the ASCII character of Enter
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print(f"Collecting samples completed. Collected {img_id} samples.")

generate_dataset()

Trying camera index 0
Successfully opened camera with index 0
Camera providing valid frames


In [3]:
from PIL import Image
import numpy as np
import os
import cv2

def train_classifier(data_dir):
    # Check if data directory exists
    if not os.path.exists(data_dir):
        print(f"Error: Data directory '{data_dir}' not found.")
        return
    
    # Get list of image files
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    if not path:
        print(f"Error: No files found in '{data_dir}'.")
        return
    
    faces = []
    ids = []
    
    for image in path:
        try:
            img = Image.open(image).convert('L')
            imageNp = np.array(img, 'uint8')
            id = int(os.path.split(image)[1].split(".")[1])
            
            faces.append(imageNp)
            ids.append(id)
            # print(f"Processing image: {os.path.basename(image)}")
        except Exception as e:
            print(f"Error processing {image}: {e}")
    
    if not faces:
        print("No valid face images found.")
        return
        
    ids = np.array(ids)
    
    # Train and save classifier
    clf = cv2.face.LBPHFaceRecognizer_create()
    print(f"Training with {len(faces)} images...")
    clf.train(faces, ids)
    clf.write("classifier.xml")
    print("Training completed. Classifier saved as 'classifier.xml'")

train_classifier("data")

Training with 200 images...
Training completed. Classifier saved as 'classifier.xml'


In [17]:
import cv2
print(cv2.__version__)
# Check if face module is available
print(dir(cv2))  # should list 'face' if contrib modules are installed correctly

4.11.0
['ACCESS_FAST', 'ACCESS_MASK', 'ACCESS_READ', 'ACCESS_RW', 'ACCESS_WRITE', 'ADAPTIVE_THRESH_GAUSSIAN_C', 'ADAPTIVE_THRESH_MEAN_C', 'AGAST_FEATURE_DETECTOR_AGAST_5_8', 'AGAST_FEATURE_DETECTOR_AGAST_7_12D', 'AGAST_FEATURE_DETECTOR_AGAST_7_12S', 'AGAST_FEATURE_DETECTOR_NONMAX_SUPPRESSION', 'AGAST_FEATURE_DETECTOR_OAST_9_16', 'AGAST_FEATURE_DETECTOR_THRESHOLD', 'AKAZE', 'AKAZE_DESCRIPTOR_KAZE', 'AKAZE_DESCRIPTOR_KAZE_UPRIGHT', 'AKAZE_DESCRIPTOR_MLDB', 'AKAZE_DESCRIPTOR_MLDB_UPRIGHT', 'AKAZE_create', 'ALGO_HINT_ACCURATE', 'ALGO_HINT_APPROX', 'ALGO_HINT_DEFAULT', 'AffineFeature', 'AffineFeature_create', 'AgastFeatureDetector', 'AgastFeatureDetector_AGAST_5_8', 'AgastFeatureDetector_AGAST_7_12d', 'AgastFeatureDetector_AGAST_7_12s', 'AgastFeatureDetector_NONMAX_SUPPRESSION', 'AgastFeatureDetector_OAST_9_16', 'AgastFeatureDetector_THRESHOLD', 'AgastFeatureDetector_create', 'Algorithm', 'AlignExposures', 'AlignMTB', 'Animation', 'AsyncArray', 'BFMatcher', 'BFMatcher_create', 'BORDER_CONST